# Sampling

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns

## Population vs Sample

The **population** is the complete dataset. It doesnt have to refer to people. Typically we dont know what the whole population is.

The **sample** is the subset of data you calculate on.

In [ ]:
# The following dataset corresponds to a set of professional evaluations of coffees
coffee = pd.read_feather('../data/coffee_ratings_full.feather')

In [ ]:
coffee.head()

In [ ]:
coffee.shape

In [ ]:
coffee.dtypes

In [ ]:
coffee['variety'] = coffee.variety.astype('category')

The 1338 observations of the coffee dataset correspond to a sample, and not to the population of the kinds of existing coffee varieties. Yet, in our particular context lets consider this dataset as our population.

We can take a sample of this *population* using the *.sample()* method.

In [ ]:
coffee_samp = coffee.sample(n=10)
coffee_samp

## Population parameters and point estimates

A **population parameter** is a calculation made on the population dataset.




In [ ]:
np.mean(coffee.aftertaste)

In [ ]:
coffee.aftertaste.mean()

A **point estimate** or sample statistic is a calculation made on the sample dataset

In [ ]:
np.mean(coffee_samp.aftertaste)

In [ ]:
coffee_samp.aftertaste.mean()

## Convenience sample

**Sample bias** is a problem caused when the sample is not representative of the population.
Collecting data by the easiest method is called *convenience sampling* and often causes sample bias.

Plotting histograms of the sample vs population helps identifying selection bias

In [ ]:
coffee_bad_samp = coffee.head(10)

In [ ]:
coffee.total_cup_points.hist(bins=np.arange(0,101, 1))

In [ ]:
coffee_samp.total_cup_points.hist(bins=np.arange(0,101, 1))

In [ ]:
coffee_bad_samp.total_cup_points.hist(bins=np.arange(0,101, 1))

The random sample seems more representative than the head one.

## Pseudo-random number generation

Random numbers cannot be known beforehand. True randomness is expensive. Pseudorandomness is a good workaround.

Pseudo-random number generation is cheap and fast.
Next random number is calculated from the previous one.
The first one is calculated from a *seed*.
All future values are always the same.##

Numpy has many number generators from different statistical distributions under numpy.random

In [ ]:
import numpy.random as random

betas = random.beta(a=2, b=2, size=5000)
betas
                   

In [ ]:
sns.histplot(data=betas)

Numpy allows us to set the seed so our code is reproducible.

In [ ]:
random.seed(42)

In [ ]:
normals = random.normal(loc=2, scale=1.5, size=2000)
sns.histplot(normals)

## Simple Random and Systematic Sampling 

### Simple Random Sampling

Its like a raffle. We take n random examples, one at a time. The pandas .sample method for instance.

In [ ]:
sample = coffee.sample(n=10)

The pandas .sample() method accepts the parameter *frac* as well, that represents the fraction of the original dataset to be included in the samplem

### Systematic Random Sampling 

Picks random samples with a fixed interval. There is no pandas implementation for this, but the .iloc[::interval] works.
The systematic random sampling is only safe when there is no pattern in the data. Sampling the whole dataset avoids problems caused by patterns in the original dataset.

In [ ]:
size = len(coffee)
sample_size = 10
interval = size//sample_size
sample_sys = coffee[::interval]

sampling the whole dataset:

In [ ]:
shuffled = coffee.sample(frac=1)

In [ ]:
shuffled

## Stratified and weighted random sampling


In [ ]:
coffee.country_of_origin.value_counts(normalize=True)

In [ ]:
coffee_sample = coffee.sample(frac=0.1, random_state=42)
coffee_sample.country_of_origin.value_counts(normalize=True)

If we care about the proportions of each category in the sample, being closer to the ones of the original population, we can group by before sampling:

In [ ]:
coffee_strat = coffee.groupby("country_of_origin").sample(frac=0.1, random_state=42)
coffee_strat.country_of_origin.value_counts(normalize=True)

If we want the same amount of elements by category:

In [ ]:
coffee_strat_eq = coffee.groupby("country_of_origin").sample(n=1, random_state=42)
coffee_strat_eq.country_of_origin.value_counts(normalize=True)

In this dataset, there are countries with only one observation, so we cannot have more than 1 per group if we dont do the sampling with Replacement.

Another way of doing sampling is taking into account weights: adding a column with weights to the dataframe and passing it to the sampling method.



## Cluster Sampling

The problem with stratified samping is we need to collect data from each group. This could be a problem in terms of time and/or money.

When collecting data is expensive, we can use **cluster sampling**

Cluster sampling uses simple random sampling to pick some subgroups and use simple random sampling on those subgroups.

Cluster sampling is an example of multistage sampling.

In [ ]:
varieties_pop = list(coffee.variety.unique())
varieties_pop

In [ ]:
# Step 1:
import random

varieties_samp = random.sample(varieties_pop, k=3)
varieties_samp

In [ ]:
# Step 2:
variety_condition = coffee.variety.isin(varieties_samp)
coffee_cluster = coffee[variety_condition]

coffee_cluster['variety'] = coffee_cluster['variety'].cat.remove_unused_categories()

In [ ]:
coffee_cluster.groupby('variety').sample(n=5, random_state=42)

### Relative Error of point estimates

How does the size of the sample impact the accuracy of the point estimate?

In [ ]:
coffee.total_cup_points.mean()

In [ ]:
coffee.total_cup_points.sample(n=10, random_state=2024).mean()

In [ ]:
coffee.total_cup_points.sample(n=100, random_state=2024).mean()

In [ ]:
coffee.total_cup_points.sample(n=1000, random_state=2024).mean()

In general, larger samples sizes gives us more accurate estimations of the populations parameter.

The **relative error** is calculated as:

$$
relativeErrorPctg = 100 * abs(populationParameter - sampleParameter) / populationParameter
$$

The relative error decreases as the sample size increases. The relative error is quite noisy too, meaning that adding or removing a couple of observations to the sample can have a huge impact in the relative error. Another property of the relative error is that it decreases to zero (when the sample size = population)



In [ ]:
popMean = coffee.total_cup_points.mean()
rel_errors = []
sample_sizes = np.arange(1, len(coffee))

for sample_size in sample_sizes:
    sample_mean = coffee.total_cup_points.sample(n=sample_size).mean()
    rel_errors.append(100*np.abs(sample_mean-popMean)/popMean)

sns.lineplot(x=sample_sizes, y=rel_errors)

## Creating a sampling distribution

In [ ]:
import matplotlib.pyplot as plt

# Create an empty list
mean_rates_5 = []
mean_rates_50 = []
mean_rates_500 = []
# Loop 500 times to create 500 sample means
for i in range(500):
	mean_rates_5.append(coffee.total_cup_points.sample(n=5).mean())
	mean_rates_50.append(coffee.total_cup_points.sample(n=50).mean())
	mean_rates_500.append(coffee.total_cup_points.sample(n=500).mean())

# Create a histogram of the 500 sample means
plt.hist(mean_rates_5, bins=20)
plt.title('Sampling distribution of the mean for samples with 5 elements')
plt.show()
plt.hist(mean_rates_50, bins=20)
plt.title('Sampling distribution of the mean for samples with 50 elements')
plt.show()
plt.hist(mean_rates_500, bins=20)
plt.title('Sampling distribution of the mean for samples with 500 elements')
plt.show()


## Approximate Sampling Distributions

Sometimes we cannot handle the exact sample distribution due to its size. In that case, we can perform an approximate sampling distribution. Its parameters can help us estimate the exact ones.


In [ ]:
import itertools

def expand_grid(data_dict):
   rows = itertools.product(*data_dict.values())
   return pd.DataFrame.from_records(rows, columns=data_dict.keys())

# Expand a grid representing 5 8-sided dice
dice = expand_grid(
  {'die1': [1, 2, 3, 4, 5, 6, 7, 8],
   'die2': [1, 2, 3, 4, 5, 6, 7, 8],
   'die3': [1, 2, 3, 4, 5, 6, 7, 8],
   'die4': [1, 2, 3, 4, 5, 6, 7, 8],
   'die5': [1, 2, 3, 4, 5, 6, 7, 8]
  })

# Add a column of mean rolls and convert to a categorical
dice['mean_roll'] = (dice['die1'] + dice['die2'] + 
                     dice['die3'] + dice['die4'] + 
                     dice['die5']) / 5
dice['mean_roll'] = dice['mean_roll'].astype('category')

# Draw a bar plot of mean_roll
dice['mean_roll'].value_counts(sort=False).plot(kind="bar")
plt.show()

In [ ]:
# Sample one to eight, five times, with replacement
five_rolls = np.random.choice(list(range(1, 9)), size=5, replace=True)

# Print the mean of five_rolls
print(five_rolls.mean())

# Replicate the sampling code 1000 times
sample_means_1000 = []
for i in range(1000):
    sample_means_1000.append(
  		np.random.choice(list(range(1, 9)), size=5, replace=True).mean()
    )
    
# Draw a histogram of sample_means_1000 with 20 bins
plt.hist(x=sample_means_1000, bins=10)
plt.show()



The shape of the approximate sampling one is pretty similar to the one of the exact.

## Standard errors and the Central Limit Theorem

The average of independent samples have approximately normal distributions.

As the sample size increases:
- The distribution of the averages gets closer to being normally distributed
- The width of the sampling distribution gets narrower
- The mean sample mean gets closer to the population mean
- The standard deviation sample mean decreases  
    -> Specify ddof=0 when calculating std() on populations and ddof=1 with samples.
- Estimate the population std by multiplying the standard deviation sample mean by the sqrt of the sample size.

The standard deviation of the sampling distribution, aka **Standard Error**

## Bootstrapping

Sampling with replacement (resampling), where each observation can be taken multiple times VS sampling without replacement



In [ ]:
coffee

In [ ]:
coffee.reset_index(inplace=True)

In [ ]:
coffee

In [ ]:
coffee_resamp = coffee.sample(frac=1, replace=True)

In [ ]:
coffee_resamp.index.value_counts()

In [ ]:
coffee_resamp.index.nunique()

In [ ]:
len(coffee)

Setting frac to 1 produces a sample of the same size as the original dataset, but, since replace is true, there could be repetitions of observations, leaving some of the observations out of the resampled dataset.

**Bootstrapping** is the opposite of sampling from a population:
When doing sampling we go from a population to a smaller sample. When doing bootstrapping, we build up a theoretical population from a sample.

Bootstrapping process:
- Make a resample **of the same size as the original sample**
- Calculate the statistic of interest for this bootstrap sample
- Repeat steps 1 and 2 many times

The resulting statistics are bootstrap statistics and they form a bootstrap distribution




In [ ]:
coffee_sample = coffee.sample(n=100)

mean_flavors_1000 = [] 
for i in range(1000):
    mean_flavors_1000.append(np.mean(coffee_sample.sample(frac=1, replace=True)['flavor']))

plt.hist(mean_flavors_1000)
plt.show()

Histogram of the bootstrap distribution of the sample mean. Close to a normal distribution. 

In [ ]:
coffee.flavor.mean()

In [ ]:
np.mean(mean_flavors_1000)

In [ ]:
coffee_sample.flavor.mean()

The bootstrap distribution mean is usually close to the sample mean. If the sample wasnt a good representation of the population, then, the bootstrap distribution mean may not be a good estimate of the population mean.
Bootstrap cannot correct any potential bias from sampling

If we calculate the sample std and the estimated population std using the bootstrap distribution (with ddof=1), we can see huge differences.

Standard error is the standard deviation of the statistic of interest.

Standard error times sqrt of the sample size estimates the population standard deviation.


TO REVIEW


## Confidence Intervals

*Values within one standard deviation of the mean* includes a large number of values from each of these distributions.

Sometimes a point estimate is not enough or not ideal for our purpose and giving a range of possible values centered in the point estimate is simply better since it will indicate how sure we are about that estimate.

In a normal distribution, if we use as a range the mean +/- 1 std (ddof=1) we see that there are plenty of observations falling outside of the range.

If we want to include 95% of the observations we could use quantiles (0.025 to 0.975)

Another technique to calculate the confidence interval, named Standard Error Method, is based on the quantiles and uses the Inverse Cumulative Distribution Function:
The Probability Density Function of a normal distribution is the well known bell curve.
The Cumulative Distribution Function is the integration of the PDF, to get the area under the bell curve.
The Inverse CDF flips the x and y axes and its implemented with:

In [ ]:
from scipy.stats import norm

quantile=0.025

norm.ppf(quantile, loc=0, scale=1)

It provides the x value in the bell curve for the indicated quantile